### MAP556, Ecole Polytechnique, 2020-21

# TP 5 - Régression pour l'approximation d'espérance conditionnelle


### Exercice 1.  Un exemple de programmation dynamique pour l'arrêt optimal - l'option "Put Bermuda".

On considère le processus stochastique à deux pas 
$$X_i = x_0 \ e^{-\frac12 \sigma^2 T_i + \sigma W_{i}}, \quad
\qquad i=0,1,2,
$$
où $x_0$ et $\sigma$ sont deux paramètres positifs, $T_i=i$, et $(W_i)_{0\le i \le 2}$ est un processus à temps discret définit comme suit: $W_0 = 0$, et 

- la v.a.  $W_{1}$ suit une loi Gaussienne $\mathcal{N}(0,T_1)$,


- $W_{2} = W_{1} + \Delta W$, où $\Delta W$ est une v.a. de loi $\mathcal{N}(0,T_2 - T_1)$ indépendante de $W_{1}$.

On remarquera que $(W_i)_{0\le i \le 2}$ est un mouvement Brownien pris aux instants $T_i$.
En particulier, c'est une chaîne de Markov par rapport à sa filtration $(\mathcal{F}_0, \mathcal{F}_1, \mathcal{F}_2)$, où $\mathcal{F}_i = \sigma(W_0, \dots, W_i)$.

Le processus $(X_i)_{0\le i \le 2}$ est appelé mouvement Brownien géometrique.


Dans la suite, le processus $X$ représente la valeur d'un actif sur un marché (par exemple, une action).
Le contrat financier _option Put Bermuda_ est un exemple de jeu à arrêt optimal: l'acheteur du contrar paye au vendeur un prix $u_0$ à l'instant $T_0$, et gagne en échange le droit de recevoir le montant $(K-X_t)^+$ à une date $t\in\{0,1,2 \}$ de son choix. 
La constante $K > 0$ est fixée dès le départ.

On peut justifier que le prix à attribuer à cette option à la date $T_0$ est donné par

$$u_0 = \sup_{\tau \in \mathcal{T}_2 } {\mathbb E}[ (K - X_\tau)^+ ] ,$$

où $\mathcal{T}_2$ est l'ensemble de temps d'arrêt à valeurs dans $\{0,T_1,T_2\}$ par rapport à la filtration $(\mathcal{F}_i)_{0\le i \le 2}$. 


On souhaite évaluer $u_0$ par Monte-Carlo. 
On rappelle l'équation de programmation dynamique pour l'arrêt optimal: en posant $Y_i = \mbox{ess}\sup_{\tau \in \mathcal{T}_2: \tau \ge i} {\mathbb E}[ (K - X_\tau)^+|\mathcal F_{i}]$, on a

$$
\begin{aligned}
&Y_2 = (K-X_2)^+
\\
&Y_i = \max \left( (K-X_i)^+, {\mathbb E}[Y_{i+1}|\mathcal{F}_i] \right)
\qquad i = 0, 1.
\end{aligned}
$$

#####  Question préliminaire: 
Montrer que $Y_i$ est de la forme $Y_i= u_i (W_i)$ pour une fonction mesurable $u_i: \mathbb{R} \to \mathbb{R}$, et écrire l'équation recursive satisfaite par les fonctions $u_i$.
En déduire que le prix de l'option Bermuda s'écrit

$$
u_0 = Y_0
= \max \left\{ (K-x_0)^+, {\mathbb E} \left[ \max\left( (K - X_1)^+, v_1(W_1) \right) \right] \right\},
$$

où $v_1(W_1) = {\mathbb E}[(K-X_2)^+|W_1]$.

$\blacktriangleright$ On note $(W_1^m,W_2^m)_{1 \le m \le M}$ un échantillon de tirages i.i.d. du couple $(W_1,W_2)$, et on pose $X_i^m = x_0 \, e^{-\frac12 \sigma^2 T_i \, + \, \sigma \, W_i^m}$ pour tout $m$.


1. __Prix _benchmark___: 	dans cette question, on utilisera la formule explicite
	
    $$
  v_1(W_1) = {\mathbb E}[(K-X_2)^+|W_1] = \mathrm{P}(W_{1}),
	$$
    
	où, pour tout $w \in \mathbb{R}$,
	$$
	\mathrm{P}(w) = K \, N(-d_2) - x \, N(-d_1)
	$$
    
	avec
	$$
	x = x_0  \, e^{-\frac12 \sigma^2 T_1 + \sigma \, w},
	\qquad d_2 = \frac{\log(x/K)}{\sigma \sqrt{T_2-T_1}} - \frac12 \sigma \sqrt{T_2-T_1},
	\qquad d_1 = d_2 + \sigma \sqrt{T_2-T_1},
	$$
    
	où $N(z) = \int_{-\infty}^z e^{-y^2/2} \frac{dy}{\sqrt{2 \pi}}$ est la fonction de répartition gaussienne. accessible via la fonction `scipy.stats.norm.cdf`.
    
    (Remarque: Dans la terminologie financière, la fonction $\mathrm{P}$ est le prix d'une option Put (non Bermuda, mais *Européen*) dans le modèle de Black-Scholes.)
    
    __1 (a)__. Coder cette formule dans la fonction putBlackScholes. Vérifier que pour les paramètres $x_0=1$, $T_1=1$, $T_2=2 $, $K=1.2$, $\sigma=0.2$, on obtient bien $\mathrm{P}( 0) \approx 0.2374$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from time import time 

In [ ]:
## Parametres
K = 1.2
x0 = 1.
sigma = 0.2
 
T2 = 2.
T1 = T2 / 2.
 
M = int(5e3)

def pos_part(x):
    return np.maximum(x, 0)

In [ ]:
###################################################
## Q1 (a): formule explicite
##         pour l'esperance conditionnelle 
###################################################
 
def putBlackScholes(x0, T1, T2, K, W1, sigma):
    """
    La fonction v_1(W1) de la question 1.(a)
    
    Autrement dit: le prix en T1 du Put Black-Scholes de maturite T2
    en fonction de la valeur courante du mouv Brownien en T1.
     
    W1: un array numpy, contenant les valeurs courantes
    du mouv Brownien en T1
    """
    sigmaSqrtDeltaT = sigma * np.sqrt(T2 - T1)

    ## Output: un array de la meme taille que W    
    ln_x = np.log(x0) + sigma*W1 - 0.5*sigma*sigma*T1
    
    d2 = (ln_x - np.log(K)) / sigmaSqrtDeltaT - 0.5*sigmaSqrtDeltaT
    d1 = d2 + sigmaSqrtDeltaT
    
    return  K*norm.cdf(-d2) - np.exp(ln_x)*norm.cdf(-d1)

__1 (b)__ S'il reste du temps après avoir abordé les questions 2-3-4 ci-dessous, démontrer la formule donnée pour la fonction $v_1$.

La connaissance explicite de la fonction $v_1$ permet d'écrire l'estimateur
$$
u_0^{M} = \max \Bigl\{
(K-x_0)^+, \frac 1M \sum_{m = 1}^M \max\left( (K - X_{1}^m)^+, \mathrm{P}(W_{1}^m) \right) \Bigr\}.
$$

__1 (c)__ Simuler cet estimateur dans le code suivant. Pour calculer le maximum $\max(a,b)$ entre deux nombres, on pourra utiliser la fonction `numpy.maximum(a,b)`.

Pour les valeurs considérées des paramètres, on peut montrer que l'on a

$$
{\mathbb E} \left[ \max\left( (K - X_1)^+, v_1(W_1) \right) \right] > (K-x_0)^+.
$$

L'estimateur $u_0^{M}$ satisfait-il un TCL?

In [ ]:
###################################################
## On genere les tirages gaussiens et on construit
## le mouv Brownien W et le mouv Brownien géometrique
## aux instants T1 et T2
###################################################
def MBG(x0, T, sigma, W):
    """
    Processus X: mouvement Brownien géometrique à
    l'instant T, de parametre de volatilite sigma,
    a partir de la valeur du mouvement Brownien W
    """
    return x0 * np.exp(-0.5*sigma*sigma*T + sigma * W)
 
time0 = time()
 
#################################################
## TO DO: Remplacer W1, W2 avec un echantillon de
## tirages du mouvement Brownien aux dates T1 et T2
## et X1, X2 avec le MBG correspondant
W1 = np.sqrt(T1) * np.random.randn(M) # gaussiennes N(0,T_1)
W2 = W1 + np.sqrt(T2-T1) * np.random.randn(M) # W1 + gaussiennes N(0,T_2-T_1) independantes

X1 = MBG(x0, T1, sigma, W1)
X2 = MBG(x0, T2, sigma, W2)
################################################
 
time0_1 = time()
 
timeSimulations = time0_1 - time0
 
###################################################
### 1. Prix Benchmark
###################################################
 
time1 = time()
 
v_1 = putBlackScholes(x0, T1, T2, K, W1, sigma)
 
################################################
## TO DO: completer avec le calcul du prix
## benchmark u_0^M et l'estimation de la variance
## asymptotique de l'estimateur
echantillon = np.maximum(pos_part(K - X1), v_1) ## echantillon de taille M

prix_benchmark = np.maximum( pos_part(K - x0), np.mean(echantillon) )

## TCL par méthode delta
var_TCL = np.var(echantillon)

################################################ 
rayonIC = 1.96 * np.sqrt(var_TCL/M)
 
time2 = time()
 
print("Prix benchmark = %1.4f +/- %1.4f \n" %(prix_benchmark, rayonIC) )
print("Erreur relative (TCL) = %1.3f \n" %(rayonIC / prix_benchmark) )
print("Time: %1.4f \n" %(time2 - time1 + timeSimulations) )

2. __Prix par régression empirique:__ 

    - On choisit comme espace d'approximation l'espace $\Phi$ engendré par les fonctions indicatrices de $n$ intervalles disjoints:
$$
\phi_k = 1_{I_k},
\quad
\mbox{où }
I_k = \left[-a + (k-1) \delta, -a + k \delta \right[;
\quad \delta = \frac{2a}n;
\quad k = 1, \dots, n
$$
$$
\Phi = \text{Vect}(\phi_1,\dots, \phi_n)
= \left\{\sum_{k=1}^n \alpha_k \phi_k(\cdot): \alpha_1, \dots, \alpha_n \in \mathbb{R} \right\}.
$$
On notera $\alpha \cdot \phi(\cdot) = \sum_{k=1}^n \alpha_k \phi_k(\cdot)$.

    - L'approximation empirique de la fonction $v_1(\cdot)$ est donnée par
$$
\tilde v_1(\cdot) = \sum_{k=1}^{n} \alpha_k^* \ 1_{I_k}(\cdot)
$$
où
$$ \label{e:regrEmp}
\begin{aligned}
\alpha^* \in \underset{\alpha\in \mathbb{R}^n}{\rm arg\min}
\ \sum_{m = 1}^M
\left((K-X_2^m)^+ - \alpha \cdot \phi(W_1^m)\right)^2.
\end{aligned}
$$
Si l'argmin n'est pas unique, on choisira le $\alpha^*$ de norme minimale dans $\mathbb{R}^n$.

L'estimateur par régression empirique du prix du Put Bermuda est maintenant
$$
\tilde{u}_0^{M} = 
\max \Bigl\{
(K - x_0)^+, \frac 1M \sum_{m=1}^M \tilde u_1(W_1^m)
\Bigr\}
$$
où $\tilde u_1(W_1^m) =  \max \left( (K - X_{1}^m)^+, \tilde{v}_1(W_1^m) \right)$.

__ 2 (a)__  Rappeler l'expression des coefficients $\alpha^*_k$, et l'utiliser pour compléter la fonction
coeffsRegressionEmpirique. Compléter le calcul de l'estimateur $\tilde{u}_0^{M}$.

In [ ]:
###################################################
### 2. Prix par regression empirique
###################################################
 
##################################
### On genere les cellules I_k
##################################

# Troncature du domaine pour N(0, T1)
a = 3.*np.sqrt(T1) 
 
## Dimension espace d'approximation 

# A partir d'estimation d'erreur sur la regression
# un bon choix de n (nombre de regresseurs) est
# n de l'ordre de M^{ d / (d+2) } où d est la dim de l'espace
# --> vérifier avec cours 6
n = int(M**(1./3))
 
intervals = np.linspace(-a, a, n+1)

####################################################
## TO DO: calculer
## - Les coefficients de regression empirique alpha
## - Le vecteur v_1_tilde(W1) de taille M
##   dans l'array approx_empirique_T1
####################################################
def regressionEmpirique(W1, W2, intervals, T2, K, x0, sigma):
    step = intervals[1] - intervals[0]
    alpha = np.zeros(intervals.size - 1)
    
    approx_empirique_T1 = np.zeros(M)
     
    for k in range(intervals.size - 1):
        leftPoint = intervals[k]
        
        insideCell = np.logical_and( leftPoint <= W1, W1 < leftPoint+step )         
        
        MBG_T2 = MBG(x0, T2, sigma, W2[insideCell])
        
        #############################################
        ## TO DO: Completer avec le calcul des
        ## coefficients alpha[k]
        number_inside_cell = np.sum(insideCell)
        
        if number_inside_cell != 0:
            alpha[k] = np.sum( pos_part(K - MBG_T2) ) / number_inside_cell
        #############################################
        
        #############################################
        ## TO DO: Completer avec le calcul de l'approximation
        ## empirique v_1_tilde(W1) 
        approx_empirique_T1[insideCell] = alpha[k]
        #############################################
     
    return alpha, approx_empirique_T1
 
time3 = time()
 
alpha, approx_empirique_T1 = regressionEmpirique(W1, W2, intervals, T2, K, x0, sigma) 

################################################
## TO DO: Completer avec le calcul du prix
## par regression empirique u_tilde_0^M 

echantillon = np.maximum(pos_part(K - X1), approx_empirique_T1) ## echantillon de taille M

prix_RegrEmp = np.maximum( pos_part(K - x0), np.mean(echantillon) )
################################################
 
time4 = time()
 
print("Prix par regression empirique = %1.4f" %prix_RegrEmp)
print("Time: %1.4f \n" %(time4 - time3 + timeSimulations))
 
######################################
## On peut afficher la vraie fonction
## v_1 et son approximation empirique
## pour comparaison
#####################################
x = np.linspace(-a, a, 100)
 
v_1_exact = putBlackScholes(x0, T1, T2, K, x, sigma)
 
plt.plot(x, v_1_exact, color="b", label="$v_1$")
 
plt.step(intervals, np.append(alpha, alpha[-1]), where="post", color="r", label=r"$\tilde{v}_1$")
 
plt.xlabel("valeurs de $W_1$", fontsize=17)
plt.legend(loc="best", fontsize=20)
 

3. __Prix _Simulations dans les simulations:___ L'estimateur est
$$
\hat{u}_0^{M} = \max \Bigl\{ (K-x_0)^+,
\frac 1M \sum_{m = 1}^M \max\left( (K - X_{1}^m)^+, \hat{v}_{1}^m \right)
\Bigr\}
$$
où pour tout $m$, $\hat{v}_{1}^m$ est obtenue à partir d'un échantillon de $M$ simulations i.i.d.
de la loi de $W_2$ conditionellement à $W_1$, notées 
$(W_2^{m,m'})_{1 \le m' \le M}$:
$$
\hat{v}_{1}^m = \frac1{M} \sum_{m' = 1}^M \bigl(K - x_0 \, e^{-\frac12 \sigma^2 T_2 + \sigma \, W_2^{m,m'}} \bigr)^+.
$$
    (a) Quelle est la loi de $W_2$ conditionelle à $W_1 = x$? Pour chaque $m$, simuler les $M$ tirages $W_2^{m, m'}$ suivant cette loi conditionnelle en $x =W_1^m$.
    
    (b) Compléter le calcul de cet estimateur dans le code ci-dessous, et comparer avec les méthodes précédentes.

In [ ]:
###################################################
#### 3. Prix "simulations dans les simulations"
###################################################
 
###################################################
## On genere M tirages de X2 pour chaque valeur
## dans l'echantillon X1.
## Il faudra donc repeter les tirages de M gaussiennes iid 
## POUR CHAQUE valeur de X1.
###################################################
sum_u_1 = 0.
 
time7 = time()
 
for m, w1 in enumerate(W1):
    G = np.random.randn(M)
     
    ###################################################
    ## To Do: completer avec
    ## - les tirages de W_2 conditionnellement a W1=w1
    ## - la mise a jour de la somme des contributions à 
    ##   la variable u_1
    W2 = w1 + np.sqrt(T2 - T1) * G
    
    ## On implemente v_1_hat
    v_1_hat = np.mean( pos_part(K - MBG(x0, T2, sigma, W2)) )
    
    max_courant = np.maximum( pos_part(K - MBG(x0, T1, sigma, w1)), v_1_hat)
    
    sum_u_1 += max_courant
    ###################################################

u_0 = np.maximum( np.maximum(K-x0,0.), sum_u_1/M )
 
time8 = time()
 
print("Prix Sim dans sim = %1.4f" %u_0)
print("Time: %1.4f" %(time8 - time7), "\n")

4. __Prix Longstaff-Schwartz:__
on construit un temps d'arrêt optimal $\tau^*$, que l'on simule ensuite à partir des tirages $(W_1^m, W_2^m)$.
Rappelons les propriétés suivantes pour le temps d'arrêt optimal:

    (a) On pose $\tau^\star = \min\{i\ge 0 : Y_i = (K - X_i)^+ \}$ et on note $(Y^\star_i)_{0\le i \le 2}$ le processus arrêté $Y^\star_i = Y_{i \wedge \tau^\star}$. Montrer que $\tau^\star \in \mathcal{T}_2$ et que

    $$
    Y^\star_i = \mathbb{E}[Y^\star_{i+1}|\mathcal{F}_i].
    $$
    (Indication: écrire $Y_{i \wedge \tau^\star} = 1_{i < \tau^\star} Y_i \, + \, 1_{i \ge \tau^\star} Y_{\tau^\star}$ et observer que sur l'événement $\{i < \tau^\star\}$, on a $Y_i = \mathbb{E}[Y_{i+1}|\mathcal{F}_i]$.)

    (b) Utiliser $(Y^\star_i)_{0\le i \le 2}$ pour montrer que $\tau^\star$ est un temps d'arrêt optimal.

On obtient l'estimateur
$$
\check{u}_0^M = \frac1{M} \sum_{m = 1}^M (K - X^{m}_{\tau_m})^+,
\qquad
\mbox{où }
\tau_m = \left\{
\begin{array}{ll}
0 & \mbox{si } (K-x_0)^+ \ge \tilde u_0^M
\\
1 & \mbox{si } (K-x_0)^+ < \tilde u_0^M \mbox{ et } (K-X_1^m)^+ \ge \tilde{u}_1(W_1^m)
\\
2 & \mbox{sinon}.
\end{array}
\right.
$$

Compléter le calcul de cet estimateur dans le code ci-dessous.

In [ ]:
###################################################
### 4. Prix Longstaff-Schwartz
###################################################
time5 = time()
 
def tempsArretOptimal(X1, approx_empirique_T1, K, x0, M):
    ## On réutilise l'approximation empirique definie
    ## plus haut, correspondant a v_1_tilde
    gain_T1 = np.maximum(K - X1, 0.)
    
    u_1_tilde = np.maximum(gain_T1, approx_empirique_T1)
     
    mean_0 = np.mean(u_1_tilde)
     
    if np.maximum((K-x0), 0.) >= mean_0:
        return np.zeros(M)
    else:
        tau = 1 * (u_1_tilde <= gain_T1) \
              + 2 * (u_1_tilde > gain_T1)
     
    return tau
 
###################################################
## To Do: completer avec le calcul
## - du temps d'arret optimal tau (echantillon tau_m)
## - de l'estimateur Longstaff-Schwartz du prix
##
tau = tempsArretOptimal(X1, approx_empirique_T1, K, x0, M)

X = np.array([x0*np.ones(M), X1, X2])

X_tau = np.zeros(M)
for m in range(M):
    X_tau[m] = X[tau[m], m]

echantillon = pos_part(K - X_tau)
    
mean_LongSchwartz = np.mean(echantillon)
###################################################
 
time6 = time()
 
print("Prix Longstaff-Schwartz = %1.4f" %mean_LongSchwartz)
print("Time: %1.4f \n" %(time6 - time5 + time4 - time3 + timeSimulations))
 